## Initialize

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

## Import the data

In [3]:
dataset = pd.read_excel('Dataset.xls')
dataset.shape

(30000, 24)

In [4]:
dataset.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


## Create X and Y

In [5]:
X = dataset.iloc[:, 0:23].values
Y = dataset.iloc[:, 23].values

In [6]:
X.shape

(30000, 23)

In [7]:
Y.shape

(30000,)

## Preprocess the Data

In [8]:
le_X = LabelEncoder()
X[:, 1] = le_X.fit_transform(X[:, 1])

In [9]:
le_X = LabelEncoder()
X[:, 2] = le_X.fit_transform(X[:, 2])

In [10]:
le_X = LabelEncoder()
X[:, 3] = le_X.fit_transform(X[:, 3])

In [11]:
ohe_X = OneHotEncoder(categorical_features = [1])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [12]:
ohe_X = OneHotEncoder(categorical_features = [2])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [13]:
ohe_X = OneHotEncoder(categorical_features = [8])
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]

In [14]:
sc_X = StandardScaler()

In [15]:
X = sc_X.fit_transform(X)

In [16]:
X[0]

array([ 1.09377971, -1.06647132, -0.10432569, -0.73837457,  1.06689977,
       -0.44275183, -0.06416291, -0.0970632 , -0.04126615,  0.81016074,
       -1.13672015, -1.24601985,  1.79456386,  1.78234817, -0.69666346,
       -0.66659873, -1.53004603, -1.48604076, -0.64250107, -0.64739923,
       -0.66799331, -0.67249727, -0.66305853, -0.65272422, -0.34194162,
       -0.22708564, -0.29680127, -0.30806256, -0.31413612, -0.29338206])

In [17]:
X.shape

(30000, 30)

In [18]:
ohe_Y = OneHotEncoder(categorical_features = [0])

In [19]:
Y = Y.reshape(len(Y), 1)

In [20]:
Y = ohe_Y.fit_transform(Y).toarray()
Y[0]

array([ 0.,  1.])

## Create Train and Test Data

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [22]:
X_train.shape

(24000, 30)

In [23]:
X_test.shape

(6000, 30)

In [24]:
Y_train.shape

(24000, 2)

In [25]:
Y_test.shape

(6000, 2)

## Create TF Classifier

In [26]:
num_features = X.shape[1]
num_features

30

In [27]:
num_classes = Y.shape[1]
num_classes

2

In [28]:
# Y = W1.X1 + W2.X2 + ... + Wn.Xn + B
# output = 1 / (1 + e ^ (- Y))
# Create Weights and Biases Variables

W = tf.Variable(tf.zeros([num_features, num_classes]))
B = tf.Variable(tf.zeros([num_classes]))

In [29]:
# Create x and y_ placeholders to hold actual data
x = tf.placeholder(tf.float32, [None, num_features])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [30]:
# Calculate y which holds the model's predicted values
Wx = tf.matmul(x, W)
y = Wx + B

In [31]:
# Create a cost function to minimize
cost_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = y_))

In [32]:
# Create an Optimizer to minimize the cost
# Use different Optimizers to compare which suits the given problem
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost_cross_entropy)

In [33]:
def trainTheData(num_steps, optimizer_to_use, batch_size):
    init = tf.global_variables_initializer()
    # initialize the global variables
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(num_steps):
            
            # Calculate the batch start index
            if batch_size == len(X_train):
                batch_start_index = 0
            elif batch_size > len(X_train):
                raise ValueError("Batch Size : " + str(batch_size) + " cannot be greater than Data Size : ", len(X_train))
            else:
                batch_start_index = (i * batch_size) % (len(X_train) - batch_size)
                
            # Calculate the batch end index
            batch_end_index = batch_size + batch_start_index
            
            # Calculate the batch X and Y Values
            batch_X_values = X_train[batch_start_index : batch_end_index]
            batch_Y_values = Y_train[batch_start_index : batch_end_index]
            
            # Create feed dict to feed it to the optimizer
            feed = {x : np.array(batch_X_values), y_ : np.array(batch_Y_values)}
            
            sess.run(optimizer_to_use, feed_dict = feed)
            
            # Print the metrics and other parameters for every 25th iteration
            if (i + 1) % 25 == 0:
                print("After " + str(i + 1) + " iterations, cost : ", sess.run(cost_cross_entropy, feed_dict = feed))
                print("W : ", sess.run(W))
                print("B : ", sess.run(B))
                print("")
        
        # Calculate the model's predicted logit values
        Y_pred_logits = sess.run(y, feed_dict = {x : np.array(X_test)})
        
        # Convert the logit values to a tensor
        Y_pred_logits_tensor = tf.convert_to_tensor(Y_pred_logits)
        
        # Apply softmax function
        apply_softmax = tf.nn.softmax(Y_pred_logits_tensor)
        
        # Calculate the model's predicted classes
        Y_pred_classes = np.argmax(sess.run(apply_softmax), axis = 1)
        
        sess.close()
    return Y_pred_classes   

In [34]:
Y_pred_classes = trainTheData(num_steps = 200, optimizer_to_use = optimizer, batch_size = len(X_train))

After 25 iterations, cost :  0.46294
W :  [[ -2.55725048e-02   2.55725048e-02]
 [  1.96826924e-02  -1.96826886e-02]
 [ -2.29861820e-03   2.29861657e-03]
 [ -1.75379664e-02   1.75379701e-02]
 [ -1.08885244e-04   1.08884196e-04]
 [  2.65040435e-03  -2.65040365e-03]
 [  2.30737422e-02  -2.30737459e-02]
 [  5.22771664e-02  -5.22771664e-02]
 [  1.10360153e-03  -1.10360235e-03]
 [  2.60542538e-02  -2.60542538e-02]
 [  4.57527265e-02  -4.57527265e-02]
 [ -2.65986491e-02   2.65986491e-02]
 [ -2.83038080e-01   2.83038080e-01]
 [ -7.50421733e-02   7.50421658e-02]
 [ -4.37878817e-02   4.37878817e-02]
 [ -1.63744334e-02   1.63744353e-02]
 [ -1.71639808e-02   1.71639845e-02]
 [ -5.99427475e-03   5.99427335e-03]
 [  4.38490883e-02  -4.38490883e-02]
 [  1.85206085e-02  -1.85206123e-02]
 [  9.94602311e-03  -9.94602311e-03]
 [  1.42447022e-03  -1.42446952e-03]
 [ -6.15971768e-03   6.15971629e-03]
 [ -8.04230012e-03   8.04229733e-03]
 [  5.53700440e-02  -5.53700440e-02]
 [  5.75915352e-02  -5.75915277e-

In [35]:
Y_test_classes = np.argmax(Y_test, axis = 1)

## Check the accuracy

In [36]:
accuracy_score(Y_test_classes, Y_pred_classes)

0.80349999999999999

In [37]:
confusion_matrix(Y_test_classes, Y_pred_classes)

array([[4509,  129],
       [1050,  312]], dtype=int64)